In [26]:
from utils import print_score
from preprocess import preprocess_fraud_data

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score, classification_report, confusion_matrix, recall_score

import pandas as pd 
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [20]:
ruta = "./data/dataset_balanceado.csv"
df = pd.read_csv(ruta)

data = preprocess_fraud_data(df)

X_train = data['X_train']
X_val = data['X_val']
X_test = data['X_test']
y_train = data['y_train']
y_val = data['y_val']
y_test = data['y_test']

**Modelo Base de Random Forest** 

In [7]:
modelo = RandomForestClassifier(random_state=42)
modelo.fit(X_train, y_train)

print_score(modelo, X_train, y_train, X_val, y_val, train=True)
print_score(modelo, X_train, y_train, X_val, y_val, train=False)

Train Result:
Accuracy Score: 99.98%
_______________________________________________
CLASSIFICATION REPORT:
                     0           1  accuracy    macro avg  weighted avg
precision     0.999833    1.000000  0.999848     0.999917      0.999849
recall        1.000000    0.998333  0.999848     0.999167      0.999848
f1-score      0.999917    0.999166  0.999848     0.999541      0.999848
support    6000.000000  600.000000  0.999848  6600.000000   6600.000000
_______________________________________________
Confusion Matrix: 
 [[6000    0]
 [   1  599]]

Validation Result:
Accuracy Score: 90.86%
_______________________________________________
CLASSIFICATION REPORT:
                     0      1  accuracy    macro avg  weighted avg
precision     0.909050    0.0  0.908636     0.454525      0.826409
recall        0.999500    0.0  0.908636     0.499750      0.908636
f1-score      0.952131    0.0  0.908636     0.476066      0.865574
support    2000.000000  200.0  0.908636  2200.000000   

> El modelo base Random Forest con sus parámetros por defecto, sufre de `sobreajuste` (overfitting), lo que significa que ha vuelto un experto en memorizar los datos de entrenamiento, pero no es capaz de detectar ningún fraude en datos que nunca ha visto, lo cual lo convierte en un modelo inútil.

In [21]:
#modelo.estimators_ es una lista de todos los árboles de decisión entrenados
profundidades = [arbol.get_depth() for arbol in modelo.estimators_]

print(f"El modelo tiene {len(profundidades)} árboles.")
print("-" * 30)
print(f"Profundidad máxima encontrada: {max(profundidades)}")
print(f"Profundidad mínima encontrada: {min(profundidades)}")
print(f"Profundidad promedio: {np.mean(profundidades):.2f}")

print("\nProfundidades de los primeros 10 árboles:")
print(profundidades[:10])

El modelo tiene 100 árboles.
------------------------------
Profundidad máxima encontrada: 33
Profundidad mínima encontrada: 21
Profundidad promedio: 25.85

Profundidades de los primeros 10 árboles:
[22, 23, 22, 22, 24, 25, 23, 23, 27, 24]


> El modelo cuenta con 100 árboles, esto árboles tienen una profundidad máxima de 33, la mínima de 21 y el promedio es de casi 26 niveles.
>
> Al no ponerle límites (max_depth=None) a los árboles, se permitió que cada árbol creciera sin control, lo que los llevó a seguir bajando y hacer preguntas muy específicas permitiendoles ser demasiados detallados, lo cual les permite aprende hasta el mínimo detalle y memorizarlo, por eso es que el modelo funciona tan bien en entrenamiento pero es pésimo en validación, porque son datos que nunca a visto, el modelo solo memoriza no sabe generalizar.

**Podar árboles:** Limitar la profundidad y que cada hoja tenga un número mínimo de ejemplos.

In [22]:
modelo_mejorado = RandomForestClassifier(max_depth=10, min_samples_leaf=10, min_samples_split=10, random_state=42)
modelo_mejorado.fit(X_train, y_train)

print_score(modelo_mejorado, X_train, y_train, X_val, y_val, train=True)
print_score(modelo_mejorado, X_train, y_train, X_val, y_val, train=False)

Train Result:
Accuracy Score: 90.91%
_______________________________________________
CLASSIFICATION REPORT:
                     0      1  accuracy    macro avg  weighted avg
precision     0.909091    0.0  0.909091     0.454545      0.826446
recall        1.000000    0.0  0.909091     0.500000      0.909091
f1-score      0.952381    0.0  0.909091     0.476190      0.865801
support    6000.000000  600.0  0.909091  6600.000000   6600.000000
_______________________________________________
Confusion Matrix: 
 [[6000    0]
 [ 600    0]]

Validation Result:
Accuracy Score: 90.91%
_______________________________________________
CLASSIFICATION REPORT:
                     0      1  accuracy    macro avg  weighted avg
precision     0.909091    0.0  0.909091     0.454545      0.826446
recall        1.000000    0.0  0.909091     0.500000      0.909091
f1-score      0.952381    0.0  0.909091     0.476190      0.865801
support    2000.000000  200.0  0.909091  2200.000000   2200.000000
_____________

> Al podar los árboles con max_depth, min_samples_leaf y min_samples_split, se ha resuelto el problema del sobreajuste, sin embargo, al hacerlo, se ha vuelto al problema inicial de nuestros datos, el desbalance de clases. El modelo actual, aunque ya no memoriza, sigue siendo incapaz de identificar el fraude, lo que lo mantiene siendo un modelo inútil.
>
> Ahora que el modelo no puede memorizar casos específicos (debido a la poda), se ve forzado a aprender reglas generales. Dado que la clase fraude, es la clase minoritaria, la estrategia por la que opta el modelo es ignorar por completo a la clase con menor cantidda e muestras, el algoritmo concluye que el costo de equivocarse al intentar predecir un fraude es mayor que el beneficio de acertar, por lo que adopta la política de siempre predecir la clase mayoritaria. 

**class_weight='balanced'**

In [16]:
modelo_balanceado = RandomForestClassifier(max_depth=10, min_samples_leaf=10, min_samples_split=10, class_weight='balanced', random_state=42)
modelo_balanceado.fit(X_train, y_train)

print_score(modelo_balanceado, X_train, y_train, X_val, y_val, train=True)
print_score(modelo_balanceado, X_train, y_train, X_val, y_val, train=False)

Train Result:
Accuracy Score: 94.68%
_______________________________________________
CLASSIFICATION REPORT:
                     0           1  accuracy    macro avg  weighted avg
precision     0.970672    0.707846  0.946818     0.839259      0.946778
recall        0.970833    0.706667  0.946818     0.838750      0.946818
f1-score      0.970752    0.707256  0.946818     0.839004      0.946798
support    6000.000000  600.000000  0.946818  6600.000000   6600.000000
_______________________________________________
Confusion Matrix: 
 [[5825  175]
 [ 176  424]]

Validation Result:
Accuracy Score: 86.32%
_______________________________________________
CLASSIFICATION REPORT:
                     0           1  accuracy    macro avg  weighted avg
precision     0.908217    0.075630  0.863182     0.491924      0.832527
recall        0.945000    0.045000  0.863182     0.495000      0.863182
f1-score      0.926244    0.056426  0.863182     0.491335      0.847169
support    2000.000000  200.000000 

> Al combinar la regularización para controlar el sobreajuste con el parámetro class_weight='balanced' para atacar el desbalanceo, el modelo ha aprendido a identificar transacciones fraudulentas, se ha pasado de un modelo que ignoraba por completo el fraude (Recall de 0.0) a uno que lo detecta, sin embargo, los resultados actuales generan muchas falsas alarmas, y todavía no es capaz de identificar correctamente todos los verdaderos positivos.

**Ajuste de hiperpárametros**

In [ ]:
rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)

param_grid = {
    'n_estimators': [100, 200],         
    'max_depth': [7, 10, 15],           
    'min_samples_leaf': [10, 15, 20], 
    'max_features': ['sqrt', 'log2']    
}

f1_scorer = make_scorer(f1_score, pos_label=1)

grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    scoring=f1_scorer,  
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

print("Mejores parámetros encontrados:")
print(grid_search.best_params_)

print("\nMejor F1-score durante la validación cruzada:")
print(grid_search.best_score_)

best_rf_model = grid_search.best_estimator_

y_pred_val = best_rf_model.predict(X_val)
print("\n--- Reporte de Clasificación en Validación con el Mejor Modelo ---")

print(classification_report(y_val, y_pred_val))
print(confusion_matrix(y_val, y_pred_val))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Mejores parámetros encontrados:
{'max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 15, 'n_estimators': 100}

Mejor F1-score durante la validación cruzada:
0.10188426898197005

--- Reporte de Clasificación en Validación con el Mejor Modelo ---
              precision    recall  f1-score   support

           0       0.91      0.88      0.89      2000
           1       0.12      0.17      0.14       200

    accuracy                           0.81      2200
   macro avg       0.51      0.52      0.52      2200
weighted avg       0.84      0.81      0.82      2200

[[1750  250]
 [ 167   33]]


In [27]:
rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)

param_grid = {
    'n_estimators': [100, 200],         
    'max_depth': [7, 10, 15],           
    'min_samples_leaf': [10, 15, 20], 
    'max_features': ['sqrt', 'log2']    
}

recall_scorer = make_scorer(recall_score, pos_label=1)

# cv=5 significa validación cruzada de 5 folds.
# n_jobs=-1 usa todos los núcleos de tu CPU para acelerar el proceso.
# verbose=2 te dará mensajes de progreso.
grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    scoring=recall_scorer,  #La métrica para decidir qué combinación es la mejor
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

print("Mejores parámetros encontrados:")
print(grid_search.best_params_)

print("\nMejor F1-score durante la validación cruzada:")
print(grid_search.best_score_)

best_rf_model = grid_search.best_estimator_

y_pred_val = best_rf_model.predict(X_val)
print("\n--- Reporte de Clasificación en Validación con el Mejor Modelo ---")

print(classification_report(y_val, y_pred_val))
print(confusion_matrix(y_val, y_pred_val))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Mejores parámetros encontrados:
{'max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 20, 'n_estimators': 100}

Mejor F1-score durante la validación cruzada:
0.12666666666666665

--- Reporte de Clasificación en Validación con el Mejor Modelo ---
              precision    recall  f1-score   support

           0       0.91      0.86      0.89      2000
           1       0.12      0.18      0.14       200

    accuracy                           0.80      2200
   macro avg       0.52      0.52      0.51      2200
weighted avg       0.84      0.80      0.82      2200

[[1721  279]
 [ 163   37]]


**SMOTE**

In [17]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Pipeline que primero aplica SMOTE y luego entrena el clasificador regularizado
pipeline_smote = ImbPipeline([
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(max_depth=10, min_samples_leaf=10, random_state=42))
])
pipeline_smote.fit(X_train, y_train)

print_score(pipeline_smote, X_train, y_train, X_val, y_val, train=True)
print_score(pipeline_smote, X_train, y_train, X_val, y_val, train=False)

Train Result:
Accuracy Score: 84.56%
_______________________________________________
CLASSIFICATION REPORT:
                     0           1  accuracy    macro avg  weighted avg
precision     0.927114    0.227568  0.845606     0.577341      0.863519
recall        0.901000    0.291667  0.845606     0.596333      0.845606
f1-score      0.913870    0.255661  0.845606     0.584766      0.854033
support    6000.000000  600.000000  0.845606  6600.000000   6600.000000
_______________________________________________
Confusion Matrix: 
 [[5406  594]
 [ 425  175]]

Validation Result:
Accuracy Score: 81.41%
_______________________________________________
CLASSIFICATION REPORT:
                     0           1  accuracy    macro avg  weighted avg
precision     0.907322    0.076923  0.814091     0.492123      0.831831
recall        0.886000    0.095000  0.814091     0.490500      0.814091
f1-score      0.896534    0.085011  0.814091     0.490773      0.822759
support    2000.000000  200.000000 

**Ajustar hiperpárametros**

In [25]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# 1. Crear el Pipeline
# El clasificador dentro del pipeline NO lleva class_weight
pipeline = ImbPipeline([
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(random_state=42))
])

# 2. Definir el Grid de Parámetros (¡justificado!)
# Basado en tu diagnóstico (árboles muy profundos), vamos a explorar rangos razonables.
# Le decimos a GridSearchCV a qué paso del pipeline pertenecen los parámetros con 'nombre_paso__'
param_grid = {
    'classifier__n_estimators': [150, 250],
    'classifier__max_depth': [8, 12, 16],        # Rango controlado, partiendo de tu mejor modelo (10)
    'classifier__min_samples_leaf': [10, 15],    # Rango controlado, partiendo de tu mejor modelo (10)
    'classifier__max_features': ['sqrt', 0.5]  # 'sqrt' es bueno, 0.5 es otra opción común
}

# 3. Configurar y Ejecutar GridSearchCV
f1_scorer = make_scorer(f1_score, pos_label=1)

grid_search_smote = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring=f1_scorer,
    cv=5, # 5-fold CV es más robusto que 3
    n_jobs=-1,
    verbose=2
)

print("Iniciando GridSearchCV con Pipeline de SMOTE...")
grid_search_smote.fit(X_train, y_train)

# 4. Analizar los Resultados
print("\nMejores parámetros para el Pipeline con SMOTE:")
print(grid_search_smote.best_params_)

print("\nMejor F1-score durante la validación cruzada:")
print(grid_search_smote.best_score_)

# 5. Evaluar el mejor modelo en validación
best_pipeline_model = grid_search_smote.best_estimator_
y_pred_val = best_pipeline_model.predict(X_val)

print("\n--- Reporte de Clasificación en Validación con el Pipeline Optimizado ---")
print(classification_report(y_val, y_pred_val))
print(confusion_matrix(y_val, y_pred_val))

Iniciando GridSearchCV con Pipeline de SMOTE...
Fitting 5 folds for each of 24 candidates, totalling 120 fits

Mejores parámetros para el Pipeline con SMOTE:
{'classifier__max_depth': 8, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 15, 'classifier__n_estimators': 150}

Mejor F1-score durante la validación cruzada:
0.0932162214437369

--- Reporte de Clasificación en Validación con el Pipeline Optimizado ---
              precision    recall  f1-score   support

           0       0.91      0.83      0.87      2000
           1       0.08      0.14      0.10       200

    accuracy                           0.77      2200
   macro avg       0.49      0.49      0.48      2200
weighted avg       0.83      0.77      0.80      2200

[[1661  339]
 [ 171   29]]


**SMOTEENN**

In [19]:
from imblearn.combine import SMOTEENN

# Define el modelo
rf = RandomForestClassifier(max_depth=10, min_samples_leaf=10, random_state=42)

# Define la técnica SMOTEENN
smote_enn = SMOTEENN(random_state=42)

# Crea el pipeline
pipeline_smoteenn = ImbPipeline([
    ('sampler', smote_enn),
    ('classifier', rf)
])

# Entrena el pipeline
pipeline_smoteenn.fit(X_train, y_train)

# Evalúa
print("--- Resultados con SMOTEENN (Combinado) ---")
y_pred_smoteenn = pipeline_smoteenn.predict(X_val)
print(classification_report(y_val, y_pred_smoteenn))
print(confusion_matrix(y_val, y_pred_smoteenn))

--- Resultados con SMOTEENN (Combinado) ---
              precision    recall  f1-score   support

           0       0.91      0.59      0.72      2000
           1       0.09      0.40      0.14       200

    accuracy                           0.57      2200
   macro avg       0.50      0.49      0.43      2200
weighted avg       0.83      0.57      0.67      2200

[[1186  814]
 [ 121   79]]


> **Conclusiones:**
>
> El modelo base demostró sobreajuste, tuvo un rendimiento perfecto en entrenamiento pero nulo en validación (Recall de 0.0), causado por árboles de una profundidad media de 26 niveles.
> 
> Se exploraron dos estrategias principales, la primera consistió en regularizar el modelo (limitando max_depth y min_samples_leaf) y aplicar un ajuste de pesos con class_weight='balanced'. Tras una optimización con GridSearchCV, este enfoque alcanzó un F1-score de 0.14 y un Recall del 17% en el conjunto de validación.